### 연습문제 7

다음 단계를 따라 moons 데이터셋에 결정 트리를 훈련시키고 세밀하게 튜닝해보세요

In [1]:
# a. make_moons(n_samples=1000, noise=0.4)를 사용해 데이터셋을 생성

from sklearn.datasets import make_moons

X, y = make_moons(n_samples =1000, noise=0.4, random_state = 42)

In [2]:
# b. train_test_split()을 사용해 훈련셋과 테스트셋으로 나누기

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [3]:
# 최적의 매개변수를 찾기 위해 교차검증, 그리드 탐색 수행

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes': list(range(2, 50)), 'min_samples_split':[5, 6, 7]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, verbose=1, cv=3)                
grid_search_cv.fit(x_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed:    0.8s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [5, 6, 7]},
             verbose=1)

- min_samples_split: 분할되기 위해 노드가 가져야 하는 최소 샘플 수
- min_samples_leaf: 리프노드가 가지고 있어야 할 최소 샘플 수
- min_weight_fraction_leaf: min_samples_leaf와 같지만 가중치가 부여된 전체 샘풀 수에서의 비율
- max_features: 각 노드에서 분할에 사용할 특성의 최대 수

min으로 시작하는 매개변수를 증가시키거나, max로 시작하는 매개변수를 감소시키면 모델에 규제가 커진다

In [4]:
# 최적의 매개변수 찾기

grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=4, min_samples_split=5, random_state=42)

In [5]:
# 테스트 세트에서 성능 측정하기

from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(x_test)
accuracy_score(y_test, y_pred)

0.855

GridSearchCV는 전체 훈련 세트로 찾은 최적의 모델을 다시 훈련시킨다. refit=False로 지정하면 다시 훈련시키지 않는다._ 그래서 별도로 모델을 훈련시키지 않고도 정확도를 평가할 수 있다.

### 연습문제 8

In [7]:
# 훈련 세트의 서브셋을 1000개 생성, 각각은 무작위로 선택된 100개의 샘플을 담고있다

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100
mini_sets=[]

# 랜덤 샘플링
rs = ShuffleSplit(n_splits=n_trees, test_size=len(x_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(x_train):
    x_mini_train = x_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((x_mini_train, y_mini_train))


In [9]:
# 각 서브셋에서 결정트리를 훈련

import numpy as np
from sklearn.base import clone

# 위에서 찾은 최적의 매개변수 사용
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (x_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(x_mini_train, y_mini_train)
    
    y_pred = tree.predict(x_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.817115

In [10]:
# 결정트리 예측을 만들고 다수로 나온 예측만 취급한다 - 사이파이 mode()이용

Y_pred = np.empty([n_trees, len(x_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(x_test)
    
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [11]:
# 훈련시킨 모델의 정확도 구하기

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.85